In [3]:
pip install confluent-kafka


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.kafka:kafka-clients:2.8.0") \
    .getOrCreate()

# Define Kafka configurations (replace with your Confluent Cloud credentials)
kafka_bootstrap_servers = 'pkc-619z3.us-east1.gcp.confluent.cloud:9092'  # e.g., 'pkc-xxxxxx.us-west-2.confluent.cloud:9092'
kafka_topic = 'weather_0'  # Replace with your Kafka topic
api_key = 'HIB5S6RIZVNEC5JA'  # Your Confluent Cloud API key
api_secret = 'muj6x7nf5eEaEkR6I5mxCotcc8LTds7Nk+XaY/cnQB3fdlXvwdRXb0D32hTBBwkB'  # Your Confluent Cloud API secret

# Set Kafka options
kafka_options = {
    "kafka.bootstrap.servers": kafka_bootstrap_servers,
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.mechanisms": "PLAIN",
    "kafka.sasl.username": api_key,
    "kafka.sasl.password": api_secret,
    "subscribe": kafka_topic,
    "kafka.consumer.poll.timeout.ms": "5000",  # Adjust the poll timeout
    "kafka.consumer.max.poll.records": "500"
}

# Read data from Kafka
df = spark.readStream \
    .format("kafka") \
    .options(**kafka_options) \
    .load()

# The data from Kafka is in binary format, so we need to decode it
#df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Process the data (example: print the first 10 rows)
# query = df.writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .option("truncate", "false") \
#     .start()

# # Wait for the stream to finish
# query.awaitTermination()


In [21]:
output_dir = "C:/Users/cks12/Data/"
query=df.writeStream.outputMode("append").format("json").option("path", output_dir).option("checkpointLocation", "C:/Users/cks12/CData/").start()

In [22]:
print(query.status)

{'message': 'Getting offsets from KafkaV2[Subscribe[weather_0]]', 'isDataAvailable': False, 'isTriggerActive': True}
